## 3.3.2 Hybrid Search Example

### 1. Hybrid Search

In [1]:
### Step1. 문서 준비
sample_documents = [
    "RAG 기술의 최신 동향",
    "최신 검색 증강 시스템 연구",
    "RAG을 활용한 챗봇 개발",
    "RAG와 Dense Passage Retrieval의 비교 연구",
    "최신 AI 논문: 검색 증강 기술",
]

### Document 객체 생성
from llama_index.core import Document
documents = [Document(text=doc) for doc in sample_documents]

In [ ]:
### Step2. 희소 검색을 위한형태소 분석기 함수 작성
from konlpy.tag import Okt
okt = Okt() # Okt 형태소 분석기 초기화
def tokenize_korean_text(text): # 문서 내용 토큰화 함수 정의
    return okt.morphs(text)  # 한국어 형태소 기반 토큰화

In [ ]:
### Step3.LLM 및 임베딩 모델 설정

## API KEY 설정
from dotenv import load_dotenv
load_dotenv()

## 모델 설정
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
Settings.llm = OpenAI(model="gpt-4o", temperature=0.5)  # 모델명은 예시
embedding_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [ ]:
### Step4. 청킹을 위한 Splitter 설정
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)

In [ ]:
### Step5. Keyword(BM25) Index 설정
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.indices.keyword_table import KeywordTableIndex
keyword_index = KeywordTableIndex.from_documents(
    documents=documents,
    text_splitter=splitter,
    extract_keyword=tokenize_korean_text,
    show_progress=True,
)
## bm25 retriever 객체 생성(keyword_index 연결)
bm25_retriever = BM25Retriever.from_defaults(
    index=keyword_index,
    similarity_top_k=2,
)


c:\Users\USER\OneDrive\Desktop\Project\llamaindex_practice\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


resource module not available on Windows


Extracting keywords from nodes: 100%|██████████| 5/5 [00:09<00:00,  1.99s/it]


In [ ]:
### Step6. 임베딩 모델을 사용 한 VectorStoreIndex 생성
from llama_index.core import VectorStoreIndex
vector_index = VectorStoreIndex.from_documents(
    documents=documents,
    text_splitter=splitter,
    embed_model=embedding_model
)
# dense retriever 객체 생성(vector_index 연결)
semantic_retriever = vector_index.as_retriever(similarity_top_k=2)


In [ ]:
### Step7. 인덱스 db 구축(필요에 따라 구현)
vector_index.storage_context.persist(persist_dir="./index/ch03_hybrid_search_vertor_storage") # 로컬 저장
keyword_index.storage_context.persist(persist_dir="./index/ch03_hybrid_search_keyword_storage") # 로컬 저장


In [ ]:
### Step8. 혼합 검색 함수 구현
def hybrid_search(query, bm25_retriever, semantic_retriever, 
                bm25_weight=0.5, semantic_weight=0.5):
    # BM25 검색 결과
    bm25_results = bm25_retriever.retrieve(query)
    bm25_scores = {node.text: node.score * bm25_weight for node in bm25_results}
    
    # Semantic 검색 결과
    semantic_results = semantic_retriever.retrieve(query)
    semantic_scores = {node.text: node.score * semantic_weight for node in semantic_results}

    # Hybrid Score 결합
    combined_scores = bm25_scores.copy()
    for node_text, score in semantic_scores.items():
        if node_text in combined_scores:
            combined_scores[node_text] += score
        else:
            combined_scores[node_text] = score

    # 스코어 정렬
    sorted_results = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_results

In [ ]:
### Step9. 쿼리 검색 수행
query = "검색 증강 생성 기술에 대해 알려주세요"
search_results = hybrid_search(query, bm25_retriever, semantic_retriever, bm25_weight=0.5, semantic_weight=0.5)

print("[Hybrid Search Results]")
for rank, (node, score) in enumerate(search_results, start=1):
    print(f"{rank}. {node} (score: {score})")

[Hybrid Search Results]
1. 최신 검색 증강 시스템 연구 (score: 0.8014721855825501)
2. 최신 AI 논문: 검색 증강 기술 (score: 0.7702755156728551)
